# Notebook to append unidad de negocio based on servicio-sentido: This is a test

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time

from importlib import reload
from Utils import TransantiagoConstants
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [3]:
DTPMDir = TransantiagoConstants.DTPMDir
dates = TransantiagoConstants.updateCurrentSSHDates()

codes_path = os.path.join(DTPMDir, 'codes_services.xlsx')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

* Pre-processing codes['TS_CODE']

In [4]:
codes.loc[:,'TS_CODE'] = codes.loc[:,'TS_CODE'].apply(str)

* To re-factor

In [5]:
date = dates[1]

* End re-factor

In [6]:
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
etapas_df = etapas_builder.loadSimplifiedEtapas()


* Pre-processing etapas_df['servicio_subida'] and merging -> this would take ~ 18 extra seconds for each date.
* Be aware that this do not consider the fact that there are missing values of 'UN' in etapas_df because TS_CODES not found in codes.

In [7]:
tic = time.clock()
print(len(etapas_df.index))

etapas_df.loc[:,'servicio_subida'] = etapas_df.loc[:,'servicio_subida'].str.replace('T','')
etapas_df.loc[:,'servicio_subida'] = etapas_df.loc[:,'servicio_subida'].str.replace('00','')
etapas_df.loc[:,'TS_CODE'] = etapas_df.loc[:,'servicio_subida'].str.split(' ').str[0]
etapas_df.loc[:,'DIRECTION'] = etapas_df.loc[:,'servicio_subida'].str.split(' ').str[1]
etapas_df.loc[:,'DIRECTION'] = etapas_df.loc[:,'DIRECTION'].str.replace('R','Ret')
etapas_df.loc[:,'DIRECTION'] = etapas_df.loc[:,'DIRECTION'].str.replace('I','Ida')
merged_etapas_df = pd.merge(etapas_df,codes, on=['TS_CODE','DIRECTION'], how='left')

toc = time.clock()
print(toc-tic)

2369991
18.100606539849025


* Little test ...

In [8]:
len(merged_etapas_df.index)

2369991

* Test <font color = 'green' > passed </font>

# Closed